In [15]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.io as pio
import webbrowser
import tempfile
import json

In [16]:
data1 = pd.read_excel("C:/Users/sroy0601/IES Task/Project Timeline - Petrojam.xlsx", sheet_name='Project tasks')
data1 = data1.iloc[7:]
data1.columns = data1.iloc[0]
data1 = data1[1:]
data1.reset_index(drop=True, inplace=True)

data2 = pd.read_excel("C:/Users/sroy0601/IES Task/Project Timeline - Petrojam.xlsx", sheet_name='Gantt Chart Table & Instruction')
data2 = data2.iloc[:, :4]

data2['Color'] = data1['Color']

data2

,Tasks,Start Date,Completion,Length,Color
0,Pre-Kick Off Plan,2025-06-02 08:00:00,2025-10-08 16:00:00,128.333333,Black Outline
1,Client Onboarding,2025-06-02 08:00:00,2025-07-04 16:00:00,32.333333,Black
2,Review Existing Case Studies,2025-06-02 08:00:00,2025-06-20 16:00:00,18.333333,Orange
3,List Initial Target Benefit Ideas,2025-06-23 08:00:00,2025-07-04 16:00:00,11.333333,Orange
4,Set Up project On MS Cloud,2025-06-02 08:00:00,2025-06-16 16:00:00,14.333333,Orange
...,...,...,...,...,...
437,Set Weekly Client Call,2027-01-22 09:00:00,2027-01-22 17:00:00,0.333333,Orange
438,Ensure correct audience Weekly Client Call,2027-01-22 09:00:00,2027-01-22 17:00:00,0.333333,Orange
439,Reporting,2027-01-25 09:00:00,2027-02-05 17:00:00,11.333333,Grey
440,Publish Executive Report every month,2027-01-25 09:00:00,2027-02-05 17:00:00,11.333333,Orange


In [32]:
# Create color mapping
color_map = {
    'Black Outline': 'white',
    'Black': 'black',
    'Orange': 'orange',
    'Grey': 'grey'
}

# Create the figure with dynamic sizing
fig = px.timeline(
    data2,
    x_start="Start Date",
    x_end="Completion",
    y="Tasks",
    color="Color",
    color_discrete_map=color_map,
    title=None,  # Remove Plotly title (will use HTML title instead)
    template="plotly_white"
)

# Calculate dimensions (50px per task)
chart_height = max(800, len(data2) * 50)
task_height = 60  # Must match chart's per-task height

# Update layout for alignment
fig.update_layout(
    yaxis={
        'categoryorder': 'array',
        'categoryarray': list(reversed(data2['Tasks'].tolist())),
        'fixedrange': False,
        'showticklabels': False  # Hide y-axis labels
    },
    # yaxis={
    #     'categoryorder': 'array',
    #     'categoryarray': list(reversed(data2['Tasks'].tolist())),
    #     'fixedrange': False
    # },
    xaxis=dict(
        side='top',  # Keep x-axis at top
        rangeslider=dict(visible=False)  
    ),
    height=chart_height,
    width=4800,
    margin=dict(l=0, r=100, t=0, b=100)  # Remove top margin
)

# Style for Black Outline tasks
fig.update_traces(
    marker_line_color='black',
    marker_line_width=2,
    selector={'marker_color': 'white'}
)

# Generate task list in reversed order (matches chart)
# tasks_in_gantt_order = list(reversed(data2['Tasks'].tolist()))
tasks_in_gantt_order = list(data2['Tasks'].tolist())

# Create custom HTML structure
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Project Gantt Chart</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body, html {{
            margin: 0;
            padding: 0;
            height: 100%;
            font-family: Arial, sans-serif;
            overflow: hidden;
            display: flex;
            flex-direction: column;
        }}
        .header {{
            padding: 10px;
            background: white;
            border-bottom: 1px solid #ddd;
            text-align: center;
            font-size: 1.5em;
        }}
        .container-wrapper {{
            display: flex;
            flex: 1;
            height: calc(100% - 50px); /* Account for header */
        }}
        .task-panel {{
            width: 300px;
            background: white;
            overflow-y: auto;
            overflow-x: hidden;
            border-right: 1px solid #ddd;
            padding-top: 25px;
        }}
        .chart-container {{
            flex: 1;
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }}
        .chart-wrapper {{
            flex: 1;
            overflow: auto;
        }}
        .task-item {{
            height: {task_height}px; /* Match chart's per-task height */
            /*height: 50px;*/
            display: flex;
            align-items: center;
            padding: 0 15px;
            border-bottom: 1px solid #eee;
            box-sizing: border-box;
            background: white;
            position: relative;
        }}
        #plotly-chart {{
            display: block;
        }}
    </style>
</head>
<body>
    <!-- Header for title -->
    <div class="header">Project Gantt Chart</div>
    
    <div class="container-wrapper">
        <!-- Left Task Panel -->
        <div class="task-panel" id="task-panel">
            {' '.join([f'<div class="task-item">{task}</div>' for task in tasks_in_gantt_order])}
        </div>
        
        <!-- Right Chart Container -->
        <div class="chart-container">
            <div class="chart-wrapper" id="chart-wrapper">
                <div id="plotly-chart"></div>
            </div>
        </div>
    </div>

    <script>
        // Inject Plotly chart
        var plotlyData = {fig.to_json()};
        Plotly.newPlot('plotly-chart', plotlyData.data, plotlyData.layout);
        
        // Get DOM elements
        const taskPanel = document.getElementById('task-panel');
        const chartWrapper = document.getElementById('chart-wrapper');
        
        // Synchronized vertical scrolling
        let isSyncing = false;
        
        taskPanel.addEventListener('scroll', function() {{
            if (isSyncing) return;
            isSyncing = true;
            chartWrapper.scrollTop = taskPanel.scrollTop;
            isSyncing = false;
        }});
        
        chartWrapper.addEventListener('scroll', function() {{
            if (isSyncing) return;
            isSyncing = true;
            taskPanel.scrollTop = chartWrapper.scrollTop;
            isSyncing = false;
        }});
    </script>
</body>
</html>
"""

# Save to temp file and open
with tempfile.NamedTemporaryFile(suffix='.html', delete=False) as tmp:
    tmp.write(html_content.encode('utf-8'))
    temp_path = tmp.name

webbrowser.open_new_tab(temp_path)
print(f"Gantt chart opened in new browser tab at: {temp_path}")

Gantt chart opened in new browser tab at: C:\Users\sroy0601\AppData\Local\Temp\tmptyoez2v2.html
